In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text


In [2]:
df1=pd.read_csv("IMDB dataset.csv")
df2=pd.read_csv("IMDB dataset.csv")

In [3]:
df1.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df1.groupby("sentiment").describe()

review                                                            \
           count unique                                                top   
sentiment                                                                    
negative   25000  24698  This show comes up with interesting locations ...   
positive   25000  24884  Loved today's show!!! It was a variety and not...   

                
          freq  
sentiment       
negative     3  
positive     5

In [5]:
# taking negative sentiment and deleting positive
df1 = df1[~df1.sentiment.str.contains("positive")]

In [6]:
df1.head()

,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative


In [7]:
# randomly picking 5000 rows
df1 = df1.sample(n=5000, replace=True)

In [8]:
df1["sentiment"].value_counts()

negative    5000
Name: sentiment, dtype: int64

In [9]:
#taking positive senitment and deleting negative
df2 = df2[~df2.sentiment.str.contains("negative")]

In [10]:
df2.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive


In [11]:
# randomly picking 5000 rows
df2 = df2.sample(n=5000, replace=True)

In [12]:
df2["sentiment"].value_counts()

positive    5000
Name: sentiment, dtype: int64

In [13]:
#concating positive and negative sentiment
frames = [df1, df2]

df = pd.concat(frames)


In [14]:
df.head()

,review,sentiment
13032,Even my five year old was bored.<br /><br />Ve...,negative
21926,Cheesy 80's horror co-starring genre favs Ken ...,negative
23623,"The director seems like a good, solid man. His...",negative
47688,This might quite possibly be the worst movie I...,negative
48132,There is a famous short story about a man who ...,negative


In [15]:
df["sentiment"].value_counts()

negative    5000
positive    5000
Name: sentiment, dtype: int64

In [16]:
class_names = ['positive', 'negative']

In [17]:
# replacing positive sentiment with 1 and negative with 0
encoding = {
    
    'positive': 1,
    'negative': 0
}

In [18]:
reviews = df['review'].values.tolist()
sentiments = df['sentiment'].tolist()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=.2)

In [20]:
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [21]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [22]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [23]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [24]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [25]:
learner.fit(2e-5,10,callbacks=[callback] )

Epoch 1/10
1334/1334 [==============================] - 9004s 7s/step - loss: 0.2824 - accuracy: 0.8829 - val_loss: 0.2870 - val_accuracy: 0.8765
Epoch 2/10
1334/1334 [==============================] - 9300s 7s/step - loss: 0.1108 - accuracy: 0.9600 - val_loss: 0.2523 - val_accuracy: 0.9105
Epoch 3/10
1334/1334 [==============================] - 9153s 7s/step - loss: 0.0538 - accuracy: 0.9818 - val_loss: 0.3186 - val_accuracy: 0.9205
Epoch 4/10
1334/1334 [==============================] - 9092s 7s/step - loss: 0.0340 - accuracy: 0.9874 - val_loss: 0.3391 - val_accuracy: 0.9230
Epoch 5/10
1334/1334 [==============================] - 9094s 7s/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 0.3387 - val_accuracy: 0.9190
Epoch 6/10
1334/1334 [==============================] - 9122s 7s/step - loss: 0.0228 - accuracy: 0.9931 - val_loss: 0.2993 - val_accuracy: 0.9185
Epoch 7/10
1334/1334 [==============================] - 9181s 7s/step - loss: 0.0226 - accuracy: 0.9925 - val_loss: 0.3646 -

In [27]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

    positive       0.95      0.86      0.90       977
    negative       0.88      0.96      0.92      1023

    accuracy                           0.91      2000
   macro avg       0.92      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



array([[842, 135],
       [ 42, 981]], dtype=int64)